In [1]:
from obspy import read, Stream, UTCDateTime
import numpy as np
from mudpy.hfsims import windowed_gaussian, apply_spectrum
from mudpy.forward import gnss_psd
import matplotlib.pyplot as plt
import h5py
from glob import glob

In [41]:
project_name = 'newfault'
GF_list = 'rc_gflist.gflist' # Station file used in fq gen code
vel_mod = 'mojave'
stas_name = GF_list.split('.')[0]
fq_dir = '/hdd/rc_fq/fall24/' + project_name + '/'
arrival_save_dir = '/hdd/rc_fq/fall24/' + project_name + '_fq_parrivals/'

ruptures = np.load('/hdd/rc_fq/fall24/' + project_name + '_ruptures.npy')
ruptures = ['newfault.003012']
test_ruptures = ['newfault.000000','newfault.000001','newfault.000002','newfault.000003','newfault.000004']

stas = np.load('/hdd/rc_fq/fall24/' + stas_name + '_station_names.npy')
stas = ['P595', 'P284']
test_stas = ['P595', 'P594', 'CCCC']
# print(stas)

In [42]:
arrivals = np.genfromtxt(arrival_save_dir + stas_name + '_arrival_times_' + 'newfault.000474' + '.csv', dtype = 'U')
k = np.where(arrivals[:,1] == 'P284')[0]
print(arrivals[k])

[['newfault.000474' 'P284' '2019-07-06T03:20:34.540000Z']]


In [43]:
data_array = np.zeros((len(ruptures)*278, 768))

mag_list = []
sta_list = []
rupt_list = []

counter = 0

for idx in range(len(ruptures)):
    
    rupt = ruptures[idx]
    
    counter += 1
    if counter % 10 == 0:
        print('Rupture ' + rupt + ' (' + str(counter) + '/' + str(len(ruptures)) + ')')
    
    log = glob(fq_dir + 'output/ruptures/' + rupt + '.log')
    log = open(log[0],'r')
    line = log.readlines()
    
    mag = str(line[15][21:27])
#     print(mag)
    
    for idx2 in range(len(stas)):
        
        sta = stas[idx2]
        
#         print(sta)
        arrivals = np.genfromtxt(arrival_save_dir + stas_name + '_arrival_times_' + rupt + '.csv', dtype = 'U')
#         print(arrivals)
            
        # Read in data 
        
        stN = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYN.sac') 
        stE = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYE.sac') 
        stZ = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYZ.sac') 
        
#         print(stN[0].stats.starttime)
        
#         stN.plot()
        
        N_data = stN[0].data
        E_data = stE[0].data
        Z_data = stZ[0].data
        
        print(N_data.shape)
        print(E_data.shape)
        print(Z_data.shape)
        
        ### Zero-pad this data ###
        
        N_data_padded = np.pad(N_data, 128, mode = 'constant')
        E_data_padded = np.pad(E_data, 128, mode = 'constant')
        Z_data_padded = np.pad(Z_data, 128, mode = 'constant')
        
        stN_pad = stN.copy()
        stN_pad[0].data = N_data_padded
        
        stE_pad = stE.copy()
        stE_pad[0].data = E_data_padded
        
        stZ_pad = stZ.copy()
        stZ_pad[0].data = Z_data_padded
        
        print(N_data_padded.shape)
        print(E_data_padded.shape)
        print(Z_data_padded.shape)
        
#         stN.plot()
#         stN_pad.plot()
#         print(stN_pad[0].stats.starttime)
        
        npts = stN_pad[0].stats.npts

        ### Trim around the arrival time ###
        
        stas_arrival = arrivals[:,1] # Station list from arrivals file
        
        i = np.where(stas_arrival == sta)[0]
        arrival = arrivals[i,2][0]
#         print('Arrival')
#         print(arrival)

        # Grab the arrival time

        arr_time = UTCDateTime(arrival)
        arr_time = arr_time + 128 # To account for padding at front
        starttime = arr_time - 128
        if rupt == 'newfault.000474' and sta == 'P284':
            endtime = arr_time + 128
        else:
            endtime = arr_time + 127 # Needs to be 256 samples, not 256 seconds

#         arr_time = arr_time + 3 # To account for padding at front
#         starttime = arr_time - 3
#         endtime = arr_time + 3
        
        print('Start time')
        print(starttime)
        print('Arrival time')
        print(arr_time)
        print('End time')
        print(endtime)
        
        print('Pad times')
#         print(stN_pad[0].times())
        print(stN_pad[0].stats)
        
        stN_trim = stN_pad.trim(starttime, endtime)
        stE_trim = stE_pad.trim(starttime, endtime)
        stZ_trim = stZ_pad.trim(starttime, endtime)
        
        stN_trim_data = stN_trim[0].data # Middle sample is the (npts + 1)/2 sample, index is (npts + 1)/2 - 1
        stE_trim_data = stE_trim[0].data
        stZ_trim_data = stZ_trim[0].data
        
        print('Trim times')
#         print(stN_trim[0].times())
        print(stN_trim[0].stats)
        
        print(stN_trim_data.shape)
        print(stE_trim_data.shape)
        print(stZ_trim_data.shape)
        
#         print(stN_trim[0].stats.starttime)
#         print(stN_trim[0].stats.endtime)
#         print(stN_trim[0].stats.npts) # Middle sample is the (npts + 1)/2 sample, index is (npts + 1)/2 - 1
#         print(len(stN_trim_data))
#         stN_trim[0].plot()

        npts = stN_trim[0].stats.npts
#         print(npts)
        arrival_idx = int((npts + 1)/2 - 1)
        
        pick_N = stN_trim_data[arrival_idx]
        pick_E = stE_trim_data[arrival_idx]   
        pick_Z = stZ_trim_data[arrival_idx]
        
#         print(pick_N)

        stN_norm = stN_trim_data - pick_N
        stE_norm = stE_trim_data - pick_E
        stZ_norm = stZ_trim_data - pick_Z    
        
        stN_zeroed = stN_norm
        stN_zeroed[0:128] = 0 # Remember that the stop index is excluded
        
        stE_zeroed = stE_norm
        stE_zeroed[0:128] = 0 
        
        stZ_zeroed = stZ_norm
        stZ_zeroed[0:128] = 0 
        
        ### Combine N, E, and Z components into one array ###
        
        comb_data = np.append(stN_zeroed, stE_zeroed)
        comb_data = np.append(comb_data, stZ_zeroed) # Order: N, E, Z
        
#         print(comb_data.shape)
        
        big_idx = idx*278 + idx2
#         print(idx, idx2, big_idx)
        try:
            data_array[big_idx] = comb_data
        except:
            print('Error: ' + str(rupt) + ', ' + str(sta))
        
        # N: indices 0 through 255 pick at index 128)
        # E: indices 256 through 511 (pick at index 384)
        # Z: indices 512 through 767 (pick at index 640)
        
#         plt.plot(comb_data) # Checking to make sure everything lines up
#         plt.xlim(635,645)
#         plt.ylim(-0.005, 0.005)
# #         plt.ylim(-0.2, 0.2)
#         plt.axvline(640)

        ### Adding new data to an array - each row = new station ### 
    
#         data_list.append(comb_data) # Add clean data instead
        
        ### Add magnitude to list
        
        rupt_list.append(str(rupt))
        sta_list.append(str(sta))
        mag_list.append(str(mag))
        

(256,)
(256,)
(256,)
(512,)
(512,)
(512,)
Start time
2019-07-06T03:19:57.871049Z
Arrival time
2019-07-06T03:22:05.871049Z
End time
2019-07-06T03:24:12.871049Z
Pad times
         network: 
         station: P595
        location: 
         channel: 
       starttime: 2019-07-06T03:19:53.040000Z
         endtime: 2019-07-06T03:28:24.040000Z
   sampling_rate: 1.0
           delta: 1.0
            npts: 512
           calib: 1.0
         _format: SAC
             sac: AttribDict({'delta': 1.0, 'depmin': -7.843602e-05, 'depmax': 6.939369e-06, 'b': 0.0, 'e': 255.0, 'stla': 35.69756, 'stlo': -117.40283, 'evla': 35.785534, 'evlo': -117.6057, 'evdp': 20.13, 'mag': 4.748, 'dist': 20.784779, 'az': 117.95009, 'baz': -61.931416, 'depmen': -2.1770076e-05, 'nzyear': 2019, 'nzjday': 187, 'nzhour': 3, 'nzmin': 19, 'nzsec': 53, 'nzmsec': 40, 'nvhdr': 6, 'npts': 256, 'iftype': 1, 'leven': 1, 'lpspol': 0, 'lovrok': 1, 'lcalda': 1, 'kstnm': 'P595'})
Trim times
         network: 
         station: P595
    